**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Production economy and CO2 taxation](#toc1_)    
- 2. [Problem 2: Career choice model](#toc2_)    
- 3. [Problem 3: Barycentric interpolation](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [982]:
# Write your code here
import numpy as np
import pandas as pd
from types import SimpleNamespace
from scipy import optimize

## 1. <a id='toc1_'></a>[Problem 1: Production economy and CO2 taxation](#toc0_)

Consider a production economy with two firms indexed by $j \in \{1,2\}$. Each produce its own good. They solve

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

Optimal firm behavior is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The implied profits are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\cdot\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single consumer supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm.<br>
She solves:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax and $T$ is lump-sum transfer. <br>
For a given $\ell$, it can be shown that optimal behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$
Such that optimal behavior is:
$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\cdot \left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$
With optimal consumption:
$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$


The government chooses $\tau$ and balances its budget so $T=\tau c_2^*$. We initially set $\tau,T=0$.

Market clearing requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$


**Question 1:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`. We choose $w=1$ as numeraire.

In [935]:
par = SimpleNamespace()

# firms
par.A = 1.0
par.gamma = 0.5

# households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0

# government
par.tau = 0.0
par.T = 0.0

# Question 3
par.kappa = 0.1

In [936]:
# Define optimal profit function
def profit(p):
    pi = (1-par.gamma)/par.gamma * (p*par.A*par.gamma)**(1/(1-par.gamma))
    return pi

#define optimal labour demand
def labour_dem(p):
    l = (p*par.A*par.gamma)**(1/(1-par.gamma))
    return l

#define optimal output
def output(p):
    y = par.A*labour_dem(p)**par.gamma
    return y

#define labour suply
def opt_labour_sup(p1, p2):
    def labour_sup(l_s):
        c1 = par.alpha*(l_s+par.T+profit(p1)+profit(p2))/p1
        c2 = (1-par.alpha)*(l_s+par.T+profit(p1)+profit(p2))/(p2+par.tau)
        l_s = np.log(c1**par.alpha * c2**(1-par.alpha))-(par.nu*l_s**(1+par.epsilon))/(1+par.epsilon)
        return - l_s

    results = optimize.minimize_scalar(labour_sup,method='bounded',bounds=(0,1000)) 
    return results.x

In [937]:
# check market clearing conditions
labour_market_clearing = []
good_1_market_clearing = []
good_2_market_clearing = []

for p1 in np.linspace(0.1,2.0,10):
    for p2 in np.linspace(0.1,2.0,10):
        l_1 = labour_dem(p1)
        l_2 = labour_dem(p2)
        l_s = opt_labour_sup(p1, p2)
        y_1 = output(p1)
        y_2 = output(p2)
        c1 = par.alpha*(l_s+par.T+profit(p1)+profit(p2))/p1
        c2 = (1-par.alpha)*(l_s+par.T+profit(p1)+profit(p2))/(p2+par.tau)
        #market clearing conditions
        if np.isclose(l_1+l_2, l_s):
            labour_market_clearing.append(True)
        else:
            labour_market_clearing.append(False)
        if np.isclose(y_1, c1):
            good_1_market_clearing.append(True)
        else:
            good_1_market_clearing.append(False)
        if np.isclose(y_2, c2):
            good_2_market_clearing.append(True)
        else:
            good_2_market_clearing.append(False)

#print results if market clearing conditions are met
if all(labour_market_clearing):
    print('Labour market clearing condition is met')
else:
    print('Labour market clearing condition is not met')

if all(good_1_market_clearing):
    print('Good 1 market clearing condition is met')
else:
    print('Good 1 market clearing condition is not met')

if all(good_2_market_clearing):
    print('Good 2 market clearing condition is met')
else:
    print('Good 2 market clearing condition is not met')

Labour market clearing condition is not met
Good 1 market clearing condition is not met
Good 2 market clearing condition is not met


**Question 2:** Find the equilibrium prices $p_1$ and $p_2$.<br>
*Hint: you can use Walras' law to only check 2 of the market clearings*

In [938]:
#def market clearing conditions
def market_clearing(p):
    l_1 = labour_dem(p[0])
    l_2 = labour_dem(p[1])
    l_s = opt_labour_sup(p[0], p[1])
    y_1 = output(p[0])
    y_2 = output(p[1])
    c1 = par.alpha*(l_s+par.T+profit(p[0])+profit(p[1]))/p[0]
    c2 = (1-par.alpha)*(l_s+par.T+profit(p[0])+profit(p[1]))/(p[1]+par.tau)
    labour_market_clearing = l_1+l_2-l_s
    good_market_1_clearing = y_1-c1
    return labour_market_clearing, good_market_1_clearing

#Using Walras' law we check if the labour market and good 1 market clears and thereby find the equlibrium prices. 
eq_results=optimize.root(market_clearing,[1.0,1.0])
equilibrium_prices=eq_results.x

print(f'Equilibrium prices are p1 = {equilibrium_prices[0]:.5f} and p2 = {equilibrium_prices[1]:.5f}')

Equilibrium prices are p1 = 0.97593 and p2 = 1.49076


Assume the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

Here $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Question 3:** What values of $\tau$ and (implied) $T$ should the government choose to maximize $SWF$?

In [939]:
# def market_clearing_tau(prices_and_tau):

#     p1, p2, tau = prices_and_tau

#     l_1 = labour_dem(p1)
#     l_2 = labour_dem(p2)
#     l_s = opt_labour_sup(p1, p2)
#     y_1 = output(p1)
#     y_2 = output(p2)
#     c1 = par.alpha*(l_s+tau*y_2+profit(p1)+profit(p2))/p1
#     c2 = (1-par.alpha)*(l_s+tau*y_2+profit(p1)+profit(p2))/(p2+tau)
#     labour_market_clearing = l_1+l_2-l_s
#     good_market_1_clearing = y_1-c1
#     good_market_2_clearing = y_2-c2
#     return labour_market_clearing, good_market_1_clearing, good_market_2_clearing

# result_tau=optimize.root(market_clearing_tau,[0.98,1.49,0.1])
# #show result with 3 decimals
# print(f'Equilibrium prices are p1 = {result_tau.x[0]:.7f}, p2 = {result_tau.x[1]:.7f} and tau = {result_tau.x[2]:.7f}')
# result_tau.x

Equilibrium prices are p1 = 1.0146595, p2 = 1.4646742 and tau = 0.1754487


array([1.0146595 , 1.46467422, 0.17544871])

In [940]:
def social_welfare(prices_and_tau):

    p1, p2, tau = prices_and_tau
    
    l_1 = labour_dem(p1)
    l_2 = labour_dem(p2)
    l_s = opt_labour_sup(p1, p2)
    y_1 = output(p1)
    y_2 = output(p2)
    c1 = par.alpha*(l_s+tau*y_2+profit(p1)+profit(p2))/p1
    c2 = (1-par.alpha)*(l_s+tau*y_2+profit(p1)+profit(p2))/(p2+tau)

    SWF = np.log(c1 ** par.alpha * c2 ** (1 - par.alpha)) - (par.nu * l_s ** (1 + par.epsilon)) / (1 + par.epsilon) - par.kappa * y_2

    if np.isclose(l_1 + l_2, l_s) and np.isclose(y_1, c1):
        return -SWF  # maximize SWF by minimizing -SWF
    else:
        return np.inf

tau_0 = [0.98,1.49,0.2]

tol=1e-5
bounds = ((tol,5.0), (tol,5.0), (0.0,1.0))
results_sfw = optimize.minimize(social_welfare,tau_0,method='SLSQP', bounds=bounds)
tau_opt = results_sfw.x
tau_opt
#show new prices and optimal tau
print(f'Equilibrium prices are p1 = {tau_opt[0]:.7f}, p2 = {tau_opt[1]:.7f} and tau = {tau_opt[2]:.7f}')

y_2_opt = output(tau_opt[1])
T = tau_opt[2]*y_2_opt   

#show optimal tau and thereby T
print(f'Tau that maximise SWF is {tau_opt[2]:.3f} and T = {T:.3f}')

Equilibrium prices are p1 = 0.9800000, p2 = 1.4900000 and tau = 0.2000000
Tau that maximise SWF is 0.200 and T = 0.149


## 2. <a id='toc2_'></a>[Problem 2: Career choice model](#toc0_)

Consider a graduate $i$ making a choice between entering $J$ different career tracks. <br>
Entering career $j$ yields utility $u^k_{ij}$. This value is unknown to the graduate ex ante, but will ex post be: <br>
$$
    u_{i,j}^k = v_{j} + \epsilon_{i,j}^k
$$

They know that $\epsilon^k_{i,j}\sim \mathcal{N}(0,\sigma^2)$, but they do not observe $\epsilon^k_{i,j}$ before making their career choice. <br>

Consider the concrete case of $J=3$ with:
$$
\begin{align*}
    v_{1} &= 1 \\
    v_{2} &= 2 \\
    v_{3} &= 3
\end{align*}
$$

If the graduates know the values of $v_j$ and the distribution of $\epsilon_{i,j}^k$, they can calculate the expected utility of each career track using simulation: <br>
$$
    \mathbb{E}\left[ u^k_{i,j}\vert v_j \right] \approx v_j + \frac{1}{K}\sum_{k=1}^K \epsilon_{i,j}^k
$$

In [1065]:
par = SimpleNamespace()
par.J = 3
par.N = 10
par.K = 5 #10000

par.F = np.arange(1,par.N+1)
par.sigma = 2

par.v = np.array([1,2,3])
par.c = 1

**Question 1:** Simulate and calculate expected utility and the average realised utility for $K=10000$ draws, for each career choice $j$.


In [1066]:
#set seed
np.random.seed(17)

#simulate epsilon
epsilon = np.random.normal(0,par.sigma,(par.J,par.K))

#expected utility
expected_utility = np.zeros(par.J)
for j in range(par.J):
    expected_utility[j] = par.v[j] + 1/par.K * np.sum(epsilon[j,:])

#realised utility
realised_utility = np.zeros((par.J, par.K))
for j in range(par.J):
    for k in range(par.K):
        realised_utility[j,k] = par.v[j] + epsilon[j,k]

average_realised_utility = np.mean(realised_utility, axis=1)
display(expected_utility)
display(average_realised_utility)


array([1.49121627, 2.44949248, 3.76261807])

array([1.49121627, 2.44949248, 3.76261807])

Now consider a new scenario: Imagine that the graduate does not know $v_j$. The *only* prior information they have on the value of each job, comes from their $F_{i}$ friends that work in each career $j$. After talking with them, they know the average utility of their friends (which includes their friends' noise term), giving them the prior expecation: <br>
$$
\tilde{u}^k_{i,j}\left( F_{i}\right) = \frac{1}{F_{i}}\sum_{f=1}^{F_{i}} \left(v_{j} + \epsilon^k_{f,j}\right), \; \epsilon^k_{f,j}\sim \mathcal{N}(0,\sigma^2)
$$
For ease of notation consider that each graduate have $F_{i}=i$ friends in each career. <br>

For $K$ times do the following: <br>
1. For each person $i$ draw $J\cdot F_i$ values of $\epsilon_{f,j}^{k}$, and calculate the prior expected utility of each career track, $\tilde{u}^k_{i,j}\left( F_{i}\right)$. <br>
Also draw their own $J$ noise terms, $\epsilon_{i,j}^k$
1. Each person $i$ chooses the career track with the highest expected utility: $$j_i^{k*}= \arg\max_{j\in{1,2\dots,J}}\left\{ \tilde{u}^k_{i,j}\left( F_{i}\right)\right\} $$
1. Store the chosen careers: $j_i^{k*}$, the prior expectation of the value of their chosen career: $\tilde{u}^k_{i,j=j_i^{k*}}\left( F_{i}\right)$, and the realized value of their chosen career track: $u^k_{i,j=j_i^{k*}}=v_{j=j_i^{k*}}+\epsilon_{i,j=j_i^{k*}}^k$.

Chosen values will be: <br>
$i\in\left\{1,2\dots,N\right\}, N=10$ <br>
$F_i = i$<br>
So there are 10 graduates. The first has 1 friend in each career, the second has 2 friends, ... the tenth has 10 friends.

**Question 2:** Simulate and visualize: For each type of graduate, $i$, the share of graduates choosing each career, the average subjective expected utility of the graduates, and the average ex post realized utility given their choice. <br>
That is, calculate and visualize: <br>
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \mathbb{I}\left\{ j=j_i^{k*} \right\}  \;\forall j\in\left\{1,2,\dots,J\right\}
\end{align*}
$$
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \tilde{u}^k_{ij=j_i^{k*}}\left( F_{i}\right)
\end{align*}
$$
And 
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} u^k_{ij=j_i^{k*}} 
\end{align*}
$$
For each graduate $i$.

In [1088]:
#set seed
np.random.seed(17)
# Draw epsilon
prior_expected_utility = np.zeros((par.N, par.J, par.K))
realised_utility = np.zeros((par.N, par.K))
for k in range(par.K):
    for i in range(par.N):
        epsilon_friends = np.random.normal(0, par.sigma, (par.F[i],par.J, par.K))
        epsilon_own = np.random.normal(0, par.sigma, (par.J, par.K))

        for j in range(par.J):
            prior_expected_utility[i,j,k] = np.mean(par.v[j] + epsilon_friends[:,j,k])
            #find best career choice
            career = np.argmax(prior_expected_utility, axis=1)

        #realised utility given career choice
        realised_utility[i,k] = par.v[career[i,k]] + epsilon_own[career[i,k],k]


In [1102]:
#share of graduates choosing career j
share = np.zeros((par.J, par.N))
for i in range(par.N):
    for j in range(par.J):
        share[j,i] = np.sum(career[i,:]==j)/par.K
    #average prior expected utility
    average_prior_expected_utility = np.mean(prior_expected_utility, axis=2)
    #average realised utility
    average_realised_utility = np.mean(realised_utility, axis=1)

#display share, average_prior_expected_utility and average_realised_utility in a table
share = share.T  # Transpose to shape (N, J)
average_prior_expected_utility = average_prior_expected_utility[:, :3]  # Assuming first 3 columns
average_realised_utility = average_realised_utility.reshape(-1, 1)  # Reshape to column vector

# Concatenate all arrays horizontally
table = np.hstack((share, average_prior_expected_utility, average_realised_utility))

# Create a pandas DataFrame for better display
column_names = [f'Share_{j+1}' for j in range(par.J)] + [f'Avg_prior_exp_u_{j+1}' for j in range(3)] + ['Avg_realised_Utility']
df = pd.DataFrame(table, columns=column_names)

# Display the table
df

,Share_1,Share_2,Share_3,Avg_prior_exp_u_1,Avg_prior_exp_u_2,Avg_prior_exp_u_3,Avg_realised_Utility
0,0.0,0.4,0.6,1.301981,3.788823,3.973929,3.206198
1,0.2,0.2,0.6,1.075408,2.111230,2.389009,1.510276
2,0.0,0.4,0.6,1.512283,2.747546,2.844305,2.161774
3,0.0,0.2,0.8,0.185173,2.492359,3.302731,4.798480
4,0.0,0.0,1.0,0.680281,1.682378,3.094527,3.009190
5,0.0,0.2,0.8,0.961072,1.674627,3.502308,1.912028
6,0.0,0.2,0.8,1.393508,2.118352,3.026889,2.927046
7,0.0,0.0,1.0,0.849778,2.665594,3.535883,1.602869
8,0.0,0.4,0.6,1.022308,2.126622,3.065122,3.369541
9,0.0,0.0,1.0,1.360733,2.011350,3.223750,3.388666


After a year of working in their career, the graduates learn $u^k_{ij}$ for their chosen job $j_i^{k*}$ perfectly. <br>
The can switch to one of the two remaining careers, for which they have the same prior as before, but it will now include a switching cost of $c$ which is known.
Their new priors can be written as: 
$$
\tilde{u}^{k,2}_{ij}\left( F_{i}\right) = \begin{cases}
            \tilde{u}^k_{ij}\left( F_{i}\right)-c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

We will set $c=1$.

Their realized utility will be: <br>
$$
u^{k,2}_{ij}= \begin{cases}
            u_{ij}^k -c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

**Question 3:** Following the same approach as in question 2, find the new optimal career choice for each $i$, $k$. Then for each $i$, calculate the average subjective expected utility from their new optimal career choice, and the ex post realized utility of that career. Also, for each $i$, calculate the share of graduates that chooses to switch careers, conditional on which career they chose in the first year. <br>

In [1108]:
# Initialize comparison results
comparison_results = np.zeros((par.N, par.K))

# Compare realised utility with adjusted prior expected utility for not chosen careers
for k in range(par.K):
    for i in range(par.N):
        chosen_career = career[i, k]
        not_chosen_utilities = np.delete(prior_expected_utility[i, :, k], chosen_career) - 1
        print(not_chosen_utilities)
        comparison_results[i, k] = np.any(realised_utility[i, k] < not_chosen_utilities)

# Display results
print("Prior Expected Utility (not chosen careers - 1):")
# for i in range(par.N):
#     for k in range(par.K):
#         chosen_career = career[i, k]
#         not_chosen_utilities = np.delete(prior_expected_utility[i, :, k], chosen_career) - 1
#         print(f"Individual {i}, Period {k}: {not_chosen_utilities}")

# print("\nRealised Utility vs Adjusted Prior Expected Utility Comparison (1 if any adjusted prior is higher):")
# print(comparison_results)


[0.55253178 4.77327786]
[1.07668836 1.64298869]
[0.86420364 0.76288353]
[-0.06503794  0.16954439]
[ 0.45472555 -0.23432374]
[-0.05027929  0.37611558]
[0.30219821 1.276053  ]
[0.32678575 2.32076921]
[-0.86605205  1.57061372]
[0.26591323 1.34379316]
[-0.38770571  1.8226694 ]
[-0.61568765  1.20917299]
[0.70611083 0.58776409]
[-1.26494521  2.60458046]
[0.27495992 1.33934349]
[-0.22234376  0.40268917]
[1.20378233 2.22187218]
[-0.64284598  0.82536413]
[-0.2728615   0.43289662]
[0.67757246 1.6183986 ]
[-0.87434482  1.207449  ]
[-0.87602616  1.23112016]
[0.18948449 0.62009147]
[-1.10588496  2.92576752]
[0.2304348  0.66762293]
[1.04848643 1.71833787]
[0.2975071  0.33959102]
[0.0296337  1.06129264]
[1.73649472 0.0424307 ]
[0.29027189 0.51040495]
[ 1.49730196 -1.8048587 ]
[ 0.18884143 -0.94910386]
[0.8712166  2.21390642]
[-0.24195464  0.08514035]
[-0.70768088  1.51123972]
[-0.40015008 -0.6959038 ]
[-0.32968237  0.56014929]
[-0.27038255  1.74719698]
[-0.05471394  1.48134478]
[0.44450186 0.99252861

In [ ]:
#share of graduates choosing career j
share = np.zeros((par.J, par.N))
for i in range(par.N):
    for j in range(par.J):
        share[j,i] = np.sum(career[i,:]==j)/par.K
    #average prior expected utility
    average_prior_expected_utility = np.mean(prior_expected_utility, axis=2)
    #average realised utility
    average_realised_utility = np.mean(realised_utility, axis=1)

#display share, average_prior_expected_utility and average_realised_utility in a table
share = share.T  # Transpose to shape (N, J)
average_prior_expected_utility = average_prior_expected_utility[:, :3]  # Assuming first 3 columns
average_realised_utility = average_realised_utility.reshape(-1, 1)  # Reshape to column vector

# Concatenate all arrays horizontally
table = np.hstack((share, average_prior_expected_utility, average_realised_utility))

# Create a pandas DataFrame for better display
column_names = [f'Share_{j+1}' for j in range(par.J)] + [f'Avg_prior_exp_u_{j+1}' for j in range(3)] + ['Avg_realised_Utility']
df = pd.DataFrame(table, columns=column_names)

# Display the table
df

In [ ]:
import numpy as np

# Assuming par.N, par.J, par.K, par.F, par.sigma, par.v are defined appropriately

# Set seed
np.random.seed(17)

# Initialize arrays
prior_expected_utility = np.zeros((par.N, par.J, par.K))
realised_utility = np.zeros((par.N, par.K))
initial_careers = np.zeros((par.N, par.K), dtype=int)  # To track initial chosen careers
updated_careers = np.zeros((par.N, par.K), dtype=int)  # To track updated careers after comparison

for k in range(par.K):
    for i in range(par.N):
        epsilon_friends = np.random.normal(0, par.sigma, (par.F[i], par.J, par.K))
        epsilon_own = np.random.normal(0, par.sigma, (par.J, par.K))

        # Calculate prior_expected_utility
        for j in range(par.J):
            prior_expected_utility[i,j,k] = np.mean(par.v[j] + epsilon_friends[:,j,k])

        # Find the best career choice
        initial_careers[i, k] = np.argmax(prior_expected_utility[i,:,k])
        chosen_career = initial_careers[i, k]

        # Calculate realised utility for chosen career
        realised_utility[i,k] = par.v[chosen_career] + epsilon_own[chosen_career,k]

        # Modify prior_expected_utility for non-chosen careers
        for j in range(par.J):
            if j != chosen_career:
                prior_expected_utility[i,j,k] -= 1

# Initialize comparison results
comparison_results = np.zeros((par.N, par.K), dtype=bool)

# Compare realised utility with adjusted prior expected utility for not chosen careers
for k in range(par.K):
    for i in range(par.N):
        chosen_career = initial_careers[i, k]
        not_chosen_utilities = np.delete(prior_expected_utility[i, :, k], chosen_career)
        
        # Check if any non-chosen career has higher adjusted prior expected utility
        comparison_results[i, k] = np.any(realised_utility[i, k] < not_chosen_utilities)

        # Update career choice if any non-chosen career has higher adjusted prior expected utility
        if comparison_results[i, k]:
            higher_utility_careers = np.where(realised_utility[i, k] < not_chosen_utilities)[0]
            best_career_index = higher_utility_careers[np.argmax(not_chosen_utilities[higher_utility_careers])]
            if best_career_index >= chosen_career:
                best_career_index += 1  # Adjust index if necessary
            updated_careers[i, k] = best_career_index
        else:
            updated_careers[i, k] = chosen_career

# Calculate average subjective expected utility and switch probability
avg_subjective_utility = np.zeros(par.N)
switch_count = np.zeros(par.N, dtype=int)

for i in range(par.N):
    for k in range(par.K):
        avg_subjective_utility[i] += prior_expected_utility[i, updated_careers[i, k], k]
        if initial_careers[i, k] != updated_careers[i, k]:
            switch_count[i] += 1

    avg_subjective_utility[i] /= par.K  # Average over K years

switch_share = switch_count / par.K  # Share of graduates that switch careers

# Display results
print("Prior Expected Utility (not chosen careers - 1):")
for i in range(par.N):
    for k in range(par.K):
        chosen_career = initial_careers[i, k]
        not_chosen_utilities = np.delete(prior_expected_utility[i, :, k], chosen_career)
        print(f"Individual {i}, Period {k}: {not_chosen_utilities}")

print("\nRealised Utility vs Adjusted Prior Expected Utility Comparison (1 if any adjusted prior is higher):")
print(comparison_results)

print("\nInitial Careers:")
print(initial_careers)

print("\nUpdated Careers:")
print(updated_careers)

print("\nAverage Subjective Expected Utility:")
print(avg_subjective_utility)

print("\nShare of Graduates that Switch Careers:")
print(switch_share)


## 3. <a id='toc3_'></a>[Problem 3: Barycentric interpolation](#toc0_)

**Problem:** We have a set of random points in the unit square,

$$
\mathcal{X} = \{(x_1,x_2)\,|\,x_1\sim\mathcal{U}(0,1),x_2\sim\mathcal{U}(0,1)\}.
$$

For these points, we know the value of some function $f(x_1,x_2)$,

$$
\mathcal{F} = \{f(x_1,x_2) \,|\, (x_1,x_2) \in \mathcal{X}\}.
$$

Now we want to approximate the value $f(y_1,y_2)$ for some  $y=(y_1,y_2)$, where $y_1\sim\mathcal{U}(0,1)$ and $y_2\sim\mathcal{U}(0,1)$.

**Building block I**

For an arbitrary triangle $ABC$ and a point $y$, define the so-called barycentric coordinates as:

$$
\begin{align*}
  r^{ABC}_1 &= \frac{(B_2-C_2)(y_1-C_1) + (C_1-B_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_2 &= \frac{(C_2-A_2)(y_1-C_1) + (A_1-C_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_3 &= 1 - r_1 - r_2.
\end{align*}
$$

If $r^{ABC}_1 \in [0,1]$, $r^{ABC}_2 \in [0,1]$, and $r^{ABC}_3 \in [0,1]$, then the point is inside the triangle.

We always have $y = r^{ABC}_1 A + r^{ABC}_2 B + r^{ABC}_3 C$.

**Building block II**

Define the following points:

$$
\begin{align*}
A&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}>y_{2}\\
B&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}<y_{2}\\
C&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}<y_{2}\\
D&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}>y_{2}.
\end{align*}
$$

**Algorithm:**

1. Compute $A$, $B$, $C$, and $D$. If not possible return `NaN`.
1. If $y$ is inside the triangle $ABC$ return $r^{ABC}_1 f(A) + r^{ABC}_2 f(B) + r^{ABC}_3 f(C)$.
1. If $y$ is inside the triangle $CDA$ return $r^{CDA}_1 f(C) + r^{CDA}_2 f(D) + r^{CDA}_3 f(A)$.
1. Return `NaN`.



**Sample:**

In [945]:
rng = np.random.default_rng(2024)

X = rng.uniform(size=(50,2))
y = rng.uniform(size=(2,))


**Questions 1:** Find $A$, $B$, $C$ and $D$. Illustrate these together with $X$, $y$ and the triangles $ABC$ and $CDA$.

In [946]:
# write your answer here

**Question 2:** Compute the barycentric coordinates of the point $y$ with respect to the triangles $ABC$ and $CDA$. Which triangle is $y$ located inside?

In [947]:
# write your answer here

Now consider the function:
$$
f(x_1,x_2) = x_1 \cdot x_2
$$

In [948]:
f = lambda x: x[0]*x[1]
F = np.array([f(x) for x in X])

**Question 3:** Compute the approximation of $f(y)$ using the full algorithm. Compare with the true value.

In [949]:
# write your answer here

**Question 4:** Repeat question 3 for all points in the set $Y$.

In [950]:
Y = [(0.2,0.2),(0.8,0.2),(0.8,0.8),(0.8,0.2),(0.5,0.5)]

In [951]:
# write your answer here